# Assignment 4

In this assignment, you will build a model to predict length of stay of 3 days or more  (as a binary classification outcome) using a sample from the Texas hospital discharge dataset. We will evaluate your model performance using a hold-out dataset that is kept from you.

In this assignment, you will generate a full analysis report, including the descriptive stats and missingness patterns, train and evaluate the model, indicate the optimal hyperparameters, explain the model, and evaluate fairness on `sex`.

Some instructions of this assignment:

1. You should use this checklist for reporting (and complete the checklist and include it as part of the submission):
[Journal of Medical Internet Research - Consolidated Reporting Guidelines for Prognostic and Diagnostic Machine Learning Modeling Studies: Development and Validation (jmir.org)](https://www.jmir.org/2023/1/e48763)

2. You can choose which modeling technique you want to use or come up with an ensemble if you want. It is up to you. You can try and compare different models.

3. In most publications you need to explain why you did not use a simpler modeling technique, so you should have an logistic regression model as a baseline to compare against.

4. You should submit the workbook, the checklist, and the model (see details below). For submission, __only submit your best model__. 

5. We have 10 bonus points to distribute among the class based on a ranking of how well your model predicts on the holdout. Bonus points 4,3,2,1 for the top four.

## Load useful packages

In [1]:
library(sdgm)
library(dplyr)
library(ggplot2)

Loading required package: rBayesianOptimization

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: DescTools

Loading required package: lightgbm

Loading required package: R6


Attaching package: ‘lightgbm’


The following object is masked from ‘package:dplyr’:

    slice


Loading required package: magrittr


Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidy

## Load the data

In [2]:
# check the help file of this dataset. It documents the binary prediction model 
# that can be trained from this dataset.
?sdgm::texas

texas {sdgm},R Documentation


In [3]:
# In this assignment, we will provide the data pre-processing steps. Please don't change code here.

# get the dataset
df<-sdgm::getdata("texas")

# create binary outcome
df$status <- ifelse(as.numeric(df$LENGTH_OF_STAY) >= 3, 1, 0)


# transform numeric predictors
df$age  <- as.numeric(df$PAT_AGE)
df$fees <- ifelse(df$CHRGS_NON_COV < 0, NA, df$CHRGS_NON_COV)


# transform categorical predictors
df$sex            <- as.factor(ifelse(df$SEX_CODE == "F", "F",
                                      ifelse(df$SEX_CODE == "M", "M", NA)))
df$ethnicity      <- as.factor(ifelse(df$ETHNICITY == "1" | df$ETHNICITY == "1.0", 1,
                                        ifelse(df$ETHNICITY == "2" | df$ETHNICITY == "2.0", 2, NA)))
df$race           <- as.factor(df$RACE)
df$location       <- as.factor(df$PAT_STATE)
df$weekday        <- as.factor(gsub("'",'',df$ADMIT_WEEKDAY))
df$risk_mortality <- as.factor(gsub("'",'',df$RISK_MORTALITY))
df$severity       <- as.factor(gsub("'",'',df$ILLNESS_SEVERITY))
df$drg            <- as.factor(gsub("'",'',df$APR_DRG))

# select predictors
vars_select <- c("status", "age", "fees", "sex", "ethnicity", "race", "weekday",
                         "location", "risk_mortality", "severity", "drg")

# transform from tibble to dataframe
full_data <- as.data.frame(df %>% select(all_of(vars_select)))

## Question 1: Descriptive analysis

Explore and describe the dataset by print summary stats. Note this question is just to let you get familiar with the dataset, there is no need to do any data preprocessing.

In [7]:
# Have a look at the data
str(full_data)
head(full_data)
summary(full_data)
table(full_data$status)

'data.frame':	745999 obs. of  11 variables:
 $ status        : num  1 1 1 0 1 1 1 1 1 1 ...
 $ age           : num  7 12 10 7 8 17 14 9 8 14 ...
 $ fees          : num  0 0 0 0 0 0 0 0 0 0 ...
 $ sex           : Factor w/ 2 levels "F","M": 1 2 1 2 2 2 1 1 1 1 ...
 $ ethnicity     : Factor w/ 2 levels "1","2": 2 2 1 2 1 1 2 2 2 2 ...
 $ race          : Factor w/ 5 levels "1","2","3","4",..: 4 4 4 4 4 4 2 2 3 3 ...
 $ weekday       : Factor w/ 7 levels "1","2","3","4",..: 4 6 7 2 2 7 3 2 1 7 ...
 $ location      : Factor w/ 9 levels "`","AR","FC",..: 7 7 7 7 7 7 7 7 7 7 ...
 $ risk_mortality: Factor w/ 5 levels "0","1","2","3",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ severity      : Factor w/ 5 levels "0","1","2","3",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ drg           : Factor w/ 316 levels "1","110","111",..: 266 266 267 267 270 269 262 265 262 262 ...


,status,age,fees,sex,ethnicity,race,weekday,location,risk_mortality,severity,drg
,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,1,7,0,F,2,4,4,TX,1,1,754
2,1,12,0,M,2,4,6,TX,1,1,754
3,1,10,0,F,1,4,7,TX,1,1,755
4,0,7,0,M,2,4,2,TX,1,1,755
5,1,8,0,M,1,4,2,TX,1,1,758
6,1,17,0,M,1,4,7,TX,1,1,757


     status            age             fees            sex        
 Min.   :0.0000   Min.   : 0.00   Min.   :     0.0   F   :419653  
 1st Qu.:0.0000   1st Qu.: 7.00   1st Qu.:     0.0   M   :277246  
 Median :1.0000   Median :13.00   Median :     0.0   NA's: 49100  
 Mean   :0.5958   Mean   :12.02   Mean   :    57.5                
 3rd Qu.:1.0000   3rd Qu.:17.00   3rd Qu.:     0.0                
 Max.   :1.0000   Max.   :26.00   Max.   :384483.0                
 NA's   :2                        NA's   :183                     
 ethnicity       race        weekday       location      risk_mortality
 1   :209940   1   :  5750   1:126620   TX     :725177   0:   761      
 2   :525521   2   : 12529   2:128426   ZZ     :  6870   1:448749      
 NA's: 10538   3   : 94060   3:122219   NM     :  4259   2:151170      
               4   :458445   4:118523   AR     :  3552   3: 98894      
               5   :174225   5:111760   OK     :  2404   4: 46425      
               NA's:   990   6: 


     0      1 
301528 444469 

In [22]:
missing_percent <- colMeans(is.na(full_data))
sort(missing_percent, decreasing = TRUE)
#or
missing_data_df <- data.frame(Variable = names(missing_percent), Percent_Missing = missing_percent)
missing_data_df <- missing_data_df[order(-missing_data_df$Percent_Missing),]
print(missing_data_df)

data_cnt <- full_data %>% select(where(is.numeric))
cor(data_cnt, use = "complete.obs")

sex      ethnicity           race           fees       location 
  6.581778e-02   1.412602e-02   1.327080e-03   2.453086e-04   1.447723e-04 
        status            age        weekday risk_mortality       severity 
  2.680969e-06   0.000000e+00   0.000000e+00   0.000000e+00   0.000000e+00 
           drg 
  0.000000e+00

                     Variable Percent_Missing
sex                       sex    6.581778e-02
ethnicity           ethnicity    1.412602e-02
race                     race    1.327080e-03
fees                     fees    2.453086e-04
location             location    1.447723e-04
status                 status    2.680969e-06
age                       age    0.000000e+00
weekday               weekday    0.000000e+00
risk_mortality risk_mortality    0.000000e+00
severity             severity    0.000000e+00
drg                       drg    0.000000e+00


,status,age,fees
status,1.00000000,0.28519545,0.02599119
age,0.28519545,1.00000000,0.01406815
fees,0.02599119,0.01406815,1.00000000


In [6]:
# define the outcome variable
voutcome <- "status"

## Question 2: Train and evaluate model, find your best model

You are free to do as many experients as you want to find your __best model__. Remember to train a __logistic regression prediction model as your baseline__.

### Question 2.1: Train and evaluate model(s)

__Important note:__ Because you'll need to select and submit your best model, it's better to save all your models when you do the experiment. It is required to save __a model file__ for each model. 

We just updated the `sdgm` package, which has `save.model(object, filename)` and `load.model(filename)` functions that will properly save all of the models.

#### Examples
best_model<-sdgm::nested.cv.bin(sdgm::cart.bestmodel.bin, full_data, voutcome)

sdgm::save.model(best_model$model, "model_best.model") # save the model

best_model_model <- sdgm::load.model("model_best.model") # load the model

__Recommendation:__ You may also want to save __a result file__ for each model like what we did in assignment 3 to help you answer the follow-up questions.

In [11]:
# Change plot size to 18 * 6
options(repr.plot.width=18, repr.plot.height=6) 

In [12]:
n_iter <- 20
train_data_split <- 0.8
model_vec <- c("lr", "cart", "rf", "lgbm", "svm") # names of models
n_repeat <- 5

In [17]:
repeated.cv <- function(full_data, voutcome, model_vec,
                        n_iter = 20, n_repeat = 15, n_cluster = 5) {
    # parallel
    cl<-parallel::makeCluster(n_cluster)
    parallel::clusterExport(cl, c("full_data", "voutcome", "model_vec", "n_iter"),envir = environment() )

    res <- parallel::parSapply(cl, seq(n_repeat), function(i) # this is the repeated loop
    {
        res_i <- sapply(model_vec, function(model) 
        {
            nested_res <- sapply(caret::createFolds(full_data[, voutcome], k=5), function(x) # this is the nested CV outer loop
            {
                # indices of train and test set in this fold
                testInds <- x
                trnInds <- setdiff(1:nrow(full_data), testInds)

                # retrieve train and test data
                train_data <- full_data[trnInds,] 
                test_data <- full_data[testInds,]

                # build the model
                if (model == "lr") {
                    best_model<- sdgm::lr.bestmodel.bin(train_data, voutcome, n_iter=n_iter)
                } else if (model == "cart") {
                    best_model<- sdgm::cart.bestmodel.bin(train_data, voutcome, n_iter=n_iter)
                } else if (model == "rf") {
                    best_model<- sdgm::rf.bestmodel.bin(train_data, voutcome, n_iter=n_iter)
                } else if (model == "lgbm") {
                    best_model<- sdgm::lgbm.bestmodel.bin(train_data, voutcome, n_iter=n_iter)
                } else if (model == "svm") {
                    best_model<- sdgm::svm.bestmodel.bin(train_data, voutcome, n_iter=n_iter)
                }

                # predict
                preds<-predict(best_model, test_data)

                # calculate and return AUC and brier score
                if (!is.null(preds))
                {
                    test_auc <- sdgm::auc(preds, test_data[,voutcome])
                    test_brier <- sdgm::brier(preds, test_data[,voutcome] ) 
                } else  {
                    test_auc<-NA
                    test_brier <- NA
                }
                c(test_auc, test_brier)
            })
            nested_cv_auc <- mean(nested_res[1,], na.rm=TRUE)
            nested_cv_brier <- mean(nested_res[2,], na.rm=TRUE)

            c(nested_cv_auc, nested_cv_brier)
        })
        res_i
    })
    parallel::stopCluster(cl)

    # -------- define dataframes to save the intermediate results ---------
    # AUC values of the repeated train/test split
    auc <- data.frame(type = rep("nested cv", n_repeat),
                          metric = rep("auc", n_repeat),
                          lr = rep(0, n_repeat),
                          cart = rep(0, n_repeat),
                          rf = rep(0, n_repeat),
                          lgbm = rep(0, n_repeat),
                          svm = rep(0, n_repeat))
    # brier scores of the repeated train/test split
    brier <- data.frame(type = rep("nested cv", n_repeat),
                            metric = rep("brier", n_repeat),
                            lr = rep(0, n_repeat),
                            cart = rep(0, n_repeat),
                            rf = rep(0, n_repeat),
                            lgbm = rep(0, n_repeat),
                            svm = rep(0, n_repeat))

    # save results
    auc$lr     <- res[1,]
    brier$lr   <- res[2,]
    auc$cart   <- res[3,]
    brier$cart <- res[4,]
    auc$rf     <- res[5,]
    brier$rf   <- res[6,]
    auc$lgbm   <- res[7,]
    brier$lgbm <- res[8,]
    auc$svm    <- res[9,]
    brier$svm  <- res[10,]

    # return results
    return(rbind(auc, brier)) 
}

In [20]:
# ============= repeated nested 5-fold CV ============= 
res.cv.df1 <- repeated.cv(full_data, voutcome, model_vec, n_iter = n_iter, n_repeat = n_repeat)
print(res.cv.df1)

# save results
saveRDS(res.cv.df1, file = "res.cv.df1.rds")

ERROR: Error in checkForRemoteErrors(val): 5 nodes produced errors; first error: missing values and NaN's not allowed if 'na.rm' is FALSE


In [ ]:
sdgm::save.model(res.cv.df1$model, "model_best.model") # save the model

best_model_model <- sdgm::load.model("model_best.model") # load the mode

In [ ]:
# Train and evaluate the model(s)
"Your answer"

### Question 2.2: Choose your best model

Do whatever you need to determine which is your best model. And sumbit your saved best model. Please rename it as `final.model.YourName.model`. 

__Note__ Please submit your model file, not result file (if applicable).

Describe your best model here:

Model: "Your answer"

With tune? (yes/no): "Your answer"

If yes, what were the hyperparameters: "Your answer"

In [ ]:
# Do whatever you need to determine which is your best model.
"Your answer"

## Question 3: Explainability

### Question 3.1: Feature importance: permute and predict

Identify the most important 3 variables using permute and predict. Using "Lecture 9 - Variable importance on COVID.ipynb" as your reference.

__Question:__ based on your results below, what are the 3 most important variables?

Your answer:

In [ ]:
# load your best model for the question below
best_model <- "Your answer" # load the model file

In [ ]:
# permute and predict based on your best model
"Your answer"

### Question 3.2: Partial Dependence Plot (PDP)

For the most important 3 variables you identified in question 6, plot their functional relationship with the output. Using "Lecture 9 - PDP and fairness examples using sdgm.ipynb" for your reference. 

In [ ]:
# PDP of variable #1
"Your answer"

In [ ]:
# PDP of variable #2
"Your answer"

In [ ]:
# PDP of variable #3
"Your answer"

### Question 3.3

If `drg` is one of your top 3 variables, answer the below question:

What are the top 3 drugs associated with the output, i.e. higher p(1)?

Your answer

## Question 4: Fairness on sex

Refer to "Lecture 9 - PDP and fairness examples using sdgm.ipynb", explore the fairness of your best model on sex.

__Question:__ Do you think your best model is fair to females? Why?

Your answer.

In [ ]:
"Your answer"

## Question 5: Report your results

Check the reporting guideline in the instruction at the top and complete the below table.

In [ ]:
# get the hyperparameters of your best model
best_model$params

In [ ]:
### START CODE HERE  (REPLACE INSTANCES OF "Your answer" with your code) ###  
### Each "Your answer" can be different.
# answer
table <- data.frame(
  my_outcome     = "Your answer",
  my_feature     = "Your answer", # Note you need to paste all predictor names into a string
  n_feature      = "Your answer",
  n_sample         = "Your answer",
  my_model       = "Your answer",
  # for best hyperparameter, I put a space holder for three, 
  # modify it based on your best model.
  best_parameter = paste(paste("Name of hyperparameter 1:", "Your answer", "; "),
                         paste("Name of hyperparameter 2:", "Your answer", "; "),
                         paste("Name of hyperparameter 3:", "Your answer"), sep = ""),
  metric         = "auc",
  eval_results = "Your answer", # AUC value of your best model
  top_3_features = paste("Your answer", "Your answer", "Your answer", sep = "; ")
)

knitr::kable(t(table), "simple")
